In [ ]:
#! wget -q -O es.txt.gz http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.es.gz
#! gunzip -k es.txt.gz
#! mkdir lines
! split -a 3 -l 100000 no_train.txt lines/lines-

! git clone https://github.com/PolyAI-LDN/conversational-datasets.git
! pip install -q -r conversational-datasets/requirements.txt

In [ ]:
! python conversational-datasets/opensubtitles/create_data.py --runner DirectRunner --sentence_files lines/lines-* --output_dir output --dataset_format JSON

In [ ]:
from glob import glob
import pandas as pd

def remove_white_space(x):
    return (x.encode('utf-8').replace(' .', '.')
            .replace(' .', '.')
            .replace(' ,', ',')
            .replace(' ?', '?')
            .replace('¿ ', '¿')
            .replace(' !', '!')
            .replace('¡ ', '¡')
            )

def jsonl_list_to_dataframe(file_list, columns=[
        'response', 'context', 'context/0', 'context/1',
        'context/2', 'context/3', 'context/4', 'context/5',
        'context/6', 'context/7', 'context/8', 'context/9'
    ]):
    """Load a list of jsonl.gz files into a pandas DataFrame."""
    return pd.concat([pd.read_json(f,
                                   orient='records', encoding='utf-8',
                                   lines=True)[columns]
                      for f in file_list], sort=False)

df = jsonl_list_to_dataframe(glob("output/train*.json"), ).dropna()
df = df.drop_duplicates()
df = df.applymap(remove_white_space)
df.head()

In [ ]:
len(df)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
df.to_csv('final_no_conv.csv', encoding = 'utf-8')